<a href="https://colab.research.google.com/github/cathay/cnn/blob/master/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import zipfile
import random
import tensorflow as tf
from shutil import copyfile
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split


In [35]:
from functools import reduce
from keras.layers.advanced_activations import LeakyReLU

def compose(*funcs):
    """Compose arbitrarily many functions, evaluated left to right.
    Reference: https://mathieularose.com/function-composition-in-python/
    """
    # return lambda x: reduce(lambda v, f: f(v), funcs, x)
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)
    else:
        raise ValueError('Composition of empty sequence not supported.')

#build YOLO model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.optimizers import RMSprop, SGD
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16

IMAGE_WIDTH=448
IMAGE_HEIGHT=448
IMAGE_CHANNELS=3

class Yolo(object):
    def __init__(self, training=True, num_classes=21, weights=None):
        '''
        Fast RCNN introduced in Faster R-CNN.
        '''
        super(Yolo, self).__init__()

        self.training = training
        self.num_classes = num_classes
        self.model = Sequential()

        self.buildModel()


    def Conv2D(self, *args, **kwargs):
        """Wrapper to set Darknet parameters for Convolution2D."""
        _kwargs = {'kernel_regularizer': l2(5e-4)}
        _kwargs['padding'] = 'valid' if kwargs.get('strides') == (2, 2) else 'same'
        _kwargs.update(kwargs)
        return Conv2D(*args, **_kwargs)

    def Conv2D_BN_Leaky(self, *args, **kwargs):
        """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
        no_bias_kwargs = {'use_bias': False}
        no_bias_kwargs.update(kwargs)
        self.model.add(Conv2D(*args, **no_bias_kwargs))
        self.model.add(BatchNormalization())
        self.model.add(LeakyReLU(alpha=0.1))
    
    def maxPooling(self):
      self.model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))

    def buildModel(self):
      self.Conv2D_BN_Leaky(64, (7, 7), strides=2, padding='same', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))
      self.maxPooling()

      self.Conv2D_BN_Leaky(192, (3, 3), strides=2, padding='same')
      self.maxPooling()
    
    def summary(self):
      self.model.summary()  

yolo = Yolo(num_classes=30)
yolo.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_230 (Conv2D)          (None, 224, 224, 64)      9408      
_________________________________________________________________
batch_normalization_25 (Batc (None, 224, 224, 64)      256       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 224, 224, 64)      0         
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_231 (Conv2D)          (None, 56, 56, 192)       110592    
_________________________________________________________________
batch_normalization_26 (Batc (None, 56, 56, 192)       768       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 56, 56, 192)     

In [30]:


model = Sequential()
Conv2D_BN_Leaky(64, (7, 7), strides=2, padding='same', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS), model)
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))

model.add(Conv2D(192, (3, 3), padding='same'))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))

model.add(Conv2D(128, (1, 1), padding='same'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Conv2D(256, (1, 1), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))

model.add(Conv2D(256, (1, 1), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(256, (1, 1), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(256, (1, 1), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(256, (1, 1), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))# 4 times
model.add(Conv2D(512, (1, 1), padding='same'))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))

model.add(Conv2D(512, (1, 1), padding='same'))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(Conv2D(512, (1, 1), padding='same'))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(Conv2D(1024, (3, 3), strides=2, padding='same'))

model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(Conv2D(1024, (3, 3), padding='same'))

model.add(Flatten())
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(units=7*7*30))

model.summary()

SyntaxError: ignored

In [0]:


## Train
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.applications.vgg16 import preprocess_input

earlystop = EarlyStopping(patience=5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)
EPOCHS=20
BATCH_SIZE=15
callbacks = [learning_rate_reduction]

history = model.fit(x_train, 
                    y_train,
                    shuffle=True,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(x_test, y_test)
                    ,callbacks=callbacks)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 7s 0us/step

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              102761472 
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_50 (Activation)   (None, 1024)              0         
_________________________________________________________________
dro

In [0]:
model.save('driver_detection_resnet_20_epoch.h5')
shutil.copyfile('driver_detection_resnet_20_epoch.h5', DRIVE_PATH + 'driver_detection_resnet_20_epoch.h5')

'/content/drive/My Drive/colab/driver_detection_resnet_20_epoch.h5'

In [0]:
#Copy prediction file to G-drive
import shutil
CATEGORY_PATH='tmp/predict' + str(index)
if os.path.exists(CATEGORY_PATH):
  shutil.rmtree(CATEGORY_PATH)
#print(test_data.head())
if not os.path.exists(CATEGORY_PATH):
  os.mkdir(CATEGORY_PATH)
  for i in range(10):
    os.mkdir("%s/c%s"% (CATEGORY_PATH,str(i)))

test_data['category'] = np.argmax(predict_df, axis=1)
for _, row in test_data.iterrows():
  #print(row['category'])
  #print(str(row['category']))
  shutil.copyfile(TEST_DIRECTORY + "/" + row['filename'], CATEGORY_PATH + "/" + 'c' + str(row['category']) + "/" + row['filename'])

ZIP_FILE_NAME='predict' + str(index)
shutil.make_archive(ZIP_FILE_NAME, 'zip', CATEGORY_PATH)
shutil.copyfile(ZIP_FILE_NAME +'.zip','/content/drive/My Drive/colab/' + ZIP_FILE_NAME + '.zip')

predict9


'/content/drive/My Drive/colab/predict9.zip'